### only modify the cell directly below this. I think I have the rest of the code set up to output the K_b and N_A at the very end. Just modify the variables below this as needed, then click "restart and run all cells". The K_b and N_A will be printed at the very bottom. I need to modify it to also output uncertainty

In [2]:
micronsPerPixel=.16 # for 40x, use .16, for 10x use .04
#framesPerSecond=5 # I'm not sure if this is necessary, it might get the fps from the file
temperatureF=72 # I'm assuming room temperature
tracerDiameter=2*10**-6 # 2 micron diameter
mainUnit=10**-6 # shouldn't need to change this
pathToFile='../030221_S7_GFP+2micron_5min.mp4' # you might need to use ./ as opposed to ../

In [3]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience
import pims
import trackpy as tp
import matplotlib.pyplot as plt
import matplotlib as mpl

In [4]:
@pims.pipeline
def gray(image):
    return image[:, :, 1]  # Take just the green channel
frames = gray(pims.Video(pathToFile))

In [ ]:
f = tp.batch(frames, 11, minmass=500);

Frame 390: 8 features


In [ ]:
t = tp.link(f, 10, memory=3) # might need to modify the 10, idk what it should be but smaller means quicker computation

In [ ]:
t1 = tp.filter_stubs(t, 20)
# Compare the number of particles in the unfiltered and filtered data.
print('Before:', t['particle'].nunique())
print('After:', t1['particle'].nunique())

In [ ]:
plt.figure()
tp.annotate(t1[t1['frame'] == 0], frames[0]);

In [8]:
#lt.figure()
#tp.plot_traj(t1);

In [ ]:
d = tp.compute_drift(t1)

In [ ]:
#d.plot()
#plt.show()

In [ ]:
tm = tp.subtract_drift(t1.copy(), d)
#ax = tp.plot_traj(tm)
#plt.show()

In [ ]:
#im = tp.imsd(tm, micronsPerPixel/1., framesPerSecond)
#fig, ax = plt.subplots()
#ax.plot(im.index, im, 'k-', alpha=0.1)  # black lines, semitransparent
#ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
#       xlabel='lag time $t$')
#ax.set_xscale('log')
#ax.set_yscale('log')

In [ ]:
em = tp.emsd(tm, micronsPerPixel/1., 5)
#fig, ax = plt.subplots()
#ax.plot(em.index, em, 'o')
#ax.set_xscale('log')
#ax.set_yscale('log')
#ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
#       xlabel='lag time $t$')
#ax.set(ylim=(1e-2, 10));

In [ ]:
plt.figure()
plt.ylabel(r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]')
plt.xlabel('lag time $t$');
fig.savefig(pathToFile[:-4]+"VarianceSquared.png")
A=tp.utils.fit_powerlaw(em).A
#print(A)

In [ ]:
# A was .773374 for ../022521a_2micron_1min.mp4
D=A/4*mainUnit**2 # diffusion constant
temperatureK=(temperatureF-32)/1.8+273.15
eta=.0009954 # I'm assuming room temperature, shouldn't make a significant difference
r=tracerDiameter/2
k_b=D/temperatureK*6*np.pi*eta*r
print("K_b: "+str(k_b))
R=8.31446261815324
n_a=R/D/6/np.pi/eta/r*temperatureK
print("N_A: "+str(n_a))